In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amazon-fashion/meta_AMAZON_FASHION.json/meta_AMAZON_FASHION.json
/kaggle/input/amazon-fashion/AMAZON_FASHION.json/AMAZON_FASHION.json


In [3]:
import pandas as pd
df = pd.read_json('/kaggle/input/amazon-fashion/AMAZON_FASHION.json/AMAZON_FASHION.json', lines=True)

In [5]:
df.head()

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image
0,5,True,"10 20, 2014",A1D4G1SNUZWQOT,7106116521,Tracy,Exactly what I needed.,perfect replacements!!,1413763200,NaN,NaN,NaN
1,2,True,"09 28, 2014",A3DDWDH9PX2YX2,7106116521,Sonja Lau,"I agree with the other review, the opening is ...","I agree with the other review, the opening is ...",1411862400,3.0,NaN,NaN
2,4,False,"08 25, 2014",A2MWC41EW7XL15,7106116521,Kathleen,Love these... I am going to order another pack...,My New 'Friends' !!,1408924800,NaN,NaN,NaN
3,2,True,"08 24, 2014",A2UH2QQ275NV45,7106116521,Jodi Stoner,too tiny an opening,Two Stars,1408838400,NaN,NaN,NaN
4,3,False,"07 27, 2014",A89F3LQADZBS5,7106116521,Alexander D.,Okay,Three Stars,1406419200,NaN,NaN,NaN


In [6]:
meta_df = pd.read_json('/kaggle/input/amazon-fashion/meta_AMAZON_FASHION.json/meta_AMAZON_FASHION.json', lines=True)
meta_df.head()

,title,brand,feature,rank,date,asin,imageURL,imageURLHighRes,description,price,also_view,also_buy,fit,details,similar_item,tech1
0,Slime Time Fall Fest [With CDROM and Collector...,Group Publishing (CO),[Product Dimensions:\n \n8....,"13,052,976inClothing,Shoesamp;Jewelry(",8.70 inches,0764443682,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,XCC Qi promise new spider snake preparing men'...,NaN,NaN,"11,654,581inClothing,Shoesamp;Jewelry(",5 star,1291691480,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Magical Things I Really Do Do Too!,Christopher Manos,[Package Dimensions:\n \n8....,"19,308,073inClothing,ShoesJewelry(",5 star,1940280001,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,[For the professional or amateur magician. Ro...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Ashes to Ashes, Oranges to Oranges",Flickerlamp Publishing,[Package Dimensions:\n \n8....,"19,734,184inClothing,ShoesJewelry(",5 star,1940735033,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Aether & Empire #1 - 2016 First Printing Comic...,NaN,[Package Dimensions:\n \n10...,"10,558,646inClothing,Shoesamp;Jewelry(",5 star,1940967805,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN,$4.50,NaN,NaN,NaN,NaN,NaN,NaN


We will consider verified ratings only here. 

In [6]:
df = df[df['verified'] == True]

Creating the ratings dataframe as we have to work with explicit rating data only.

In [7]:
ratings = df[['reviewerID','asin','overall']]
ratings.head()

,reviewerID,asin,overall
0,A1D4G1SNUZWQOT,7106116521,5
1,A3DDWDH9PX2YX2,7106116521,2
3,A2UH2QQ275NV45,7106116521,2
5,A29HLOUW0NS0EH,7106116521,5
6,A7QS961ROI6E0,7106116521,4


In [8]:
ratings.isnull().sum()

reviewerID    0
asin          0
overall       0
dtype: int64

we will consider a rating given to a item by a user only once, same item rated by same user more than once will be dropped.

In [9]:
ratings = ratings.drop_duplicates(subset = ['reviewerID', 'asin'])

In [10]:
user_counts = ratings["reviewerID"].value_counts()

Due to resource constraints we can not work with the whole datapoints. so we are applying filter data.

In [11]:
# have rated alteast 3 items
active_users = user_counts.loc[user_counts >= 3].index.tolist()

In [12]:
ratings_active = ratings.loc[ratings.reviewerID.isin(active_users)]
print(ratings_active.shape)
print(ratings.shape)

(66294, 3)
(820759, 3)


In [13]:
n_users = len(ratings_active.reviewerID.unique())
n_item = len(ratings_active.asin.unique())
print('Unique Users: {0}, unique recipe: {1}'.format(n_users, n_item))

Unique Users: 17573, unique recipe: 32235


In [14]:
ratings_active = ratings_active.drop_duplicates(subset = ['reviewerID', 'asin'])


In [15]:
user_item_active = ratings_active.pivot('reviewerID','asin','overall').fillna(0)
print(f'Shape: {user_item_active.shape}')

Shape: (17573, 32235)


## User-User based Collaborative filtering

In [21]:
from sklearn.metrics.pairwise import cosine_similarity
# cosine similarity between users
X_user = cosine_similarity(user_item_active) # user similarity matrix

In [22]:
def item_recommender(user_item_active, X_user, user, k=20, top_n=10):
    user_ix = user_item_active.index.get_loc(user)
    print(user_ix)
    
    user_similarities = X_user[user_ix]
    print(user_similarities)
    
    most_similar_users = user_item_active.index[user_similarities.argpartition(-k)[-k:]]
    print(f"the most similar users \n {most_similar_users}")
    
    recommend_item = user_item_active.loc[most_similar_users].mean(0).sort_values(ascending=False)
    
    m_rated_item = user_item_active.loc[user].gt(0)
    print(f"rated recipes \n {m_rated_item}")
    
    rated_item = m_rated_item.index[m_rated_item].tolist()
    print(f"recorded item \n {rated_item}")
    
    recommend_item_top = recommend_item.drop(rated_item).head(top_n)
    return recommend_item_top.index.to_frame().reset_index(drop=True).merge(df)

In [23]:
recommendation = item_recommender(user_item_active, X_user, user = 'A1RRX286ZRI830')

3590
[0. 0. 0. ... 0. 0. 0.]
the most similar users 
 Index(['A20NO8IQAGW47S', 'ATFT7KSLWULGN', 'A1V2WOARN717UP', 'A2CPRQPK9A6L3Y',
       'A3LQ4070M00IOA', 'A3HC16WLSQGP15', 'A3CP5LFOXXFCTA', 'A3IBKSIKFGGV35',
       'A2VFJD55YRIIQZ', 'A30HT5C1QMNIS4', 'AT7XMEDG3S6NN', 'A1SRU3K3H30TQP',
       'A1RRX286ZRI830', 'A3TJLAAO12QQ3X', 'A2EPJ61GOO8BYT', 'A2KLBWU6KOMYVC',
       'A2VTBAX1AJ1C37', 'AIETVJQZU5ZZG', 'A3T05XA3O55RED', 'A2A7P48AU0G5BU'],
      dtype='object', name='reviewerID')
rated recipes 
 asin
1291691480    False
7106116521    False
B00007GDFV    False
B00008JOQI    False
B00008JPRZ    False
              ...  
B01HJEOBUO    False
B01HJEOC9E    False
B01HJFD866    False
B01HJG5NLI    False
B01HJHTH5U    False
Name: A1RRX286ZRI830, Length: 32235, dtype: bool
recorded item 
 ['B00515QO4K', 'B0079WLWP4', 'B008YOQ742', 'B00CHNXP0K', 'B00CHZL5J6', 'B00E9LR1AS', 'B00IWM8JHO', 'B00JE49KPO', 'B00KYH9SHU', 'B00LVYEIPW', 'B00MWWS6VE', 'B00N86PH88', 'B00NP254JW', 'B00PFNKKFM', 'B00PV9XN

In [24]:
print(recommendation)

           asin  overall  verified   reviewTime      reviewerID  \
0    B00BOBCMUU        3      True  12 12, 2015   AJIFD6Q7F62CP   
1    B00BOBCMUU        3      True   12 9, 2015   AF4RRFKOKCN3K   
2    B00BOBCMUU        3      True   12 5, 2015   AREJ8ZG9FG1ID   
3    B00BOBCMUU        5      True   12 3, 2015  A2X2BPQRSNU4T6   
4    B00BOBCMUU        5      True  11 21, 2015  A3B7K6KS0GUBCL   
..          ...      ...       ...          ...             ...   
878  B018427RWY        5      True  05 15, 2018  A2Y819EYLIXWUM   
879  B018427RWY        5      True   05 3, 2018  A10PV5WB2YRAGM   
880  B018427RWY        4      True   04 5, 2018  A1BKO2N8JUEICR   
881  B018427RWY        5      True   04 3, 2018  A20NO8IQAGW47S   
882  B018427RWY        5      True  03 11, 2018   AO88LKDEWZOGK   

         reviewerName                                         reviewText  \
0                 ADF  It is soft and keeps you warm----too bad it is...   
1    Justin Pleasants                     n

In [25]:
recommendation = item_recommender(user_item_active, X_user, user = 'A20NO8IQAGW47S')

4737
[0. 0. 0. ... 0. 0. 0.]
the most similar users 
 Index(['A29KI3D25ANRZT', 'A29KHN408YOBQX', 'A29B8ASVI73P3A', 'A29JIMDWF6VRPN',
       'A29KKT6WBXAIIO', 'A29K85C92Z8HGR', 'A29CGSOLKG6IRU', 'A29I8KIRWJDPT8',
       'A29BJWNV8WQVV0', 'A29J8Y1B05O9Q5', 'A29IS8EGZF435M', 'AEES5AZYBOKSP',
       'A29IF7O5FIW9FA', 'A2TROYM0VW0RBE', 'A1RRX286ZRI830', 'A38LSF6B5ET568',
       'A2A133Y3SWMAGR', 'A3PYX8F6P8KSD1', 'ATFT7KSLWULGN', 'A20NO8IQAGW47S'],
      dtype='object', name='reviewerID')
rated recipes 
 asin
1291691480    False
7106116521    False
B00007GDFV    False
B00008JOQI    False
B00008JPRZ    False
              ...  
B01HJEOBUO    False
B01HJEOC9E    False
B01HJFD866    False
B01HJG5NLI    False
B01HJHTH5U    False
Name: A20NO8IQAGW47S, Length: 32235, dtype: bool
recorded item 
 ['B008YOQ742', 'B00W6FSW02', 'B0179IYOMK', 'B018427RWY']


In [26]:
print(recommendation)

           asin  overall  verified   reviewTime      reviewerID  \
0    B00KA4A6Q4        1      True  09 10, 2014  A2AN6PG4BA3I7N   
1    B00KA4A6Q4        1      True   01 7, 2016   ALCX9HFB8NMD5   
2    B00KA4A6Q4        4      True  12 17, 2015   AAO3VUCRCSCJ3   
3    B00KA4A6Q4        5      True  11 16, 2015  A349OFPZ0B9K29   
4    B00KA4A6Q4        5      True  09 21, 2015  A2I1HTCC2ELG2V   
..          ...      ...       ...          ...             ...   
718  B01ETKKP60        1      True   06 4, 2018  A2FEVRYTERU4G1   
719  B01ETKKP60        5      True  05 11, 2018   AWJHDUYZE2AJ2   
720  B01ETKKP60        5      True  04 29, 2018  A3043US8LEEVGN   
721  B01ETKKP60        3      True  04 24, 2018  A2CZ6XSEBJPYEH   
722  B01ETKKP60        3      True  03 22, 2018   AE6IXSHB614KT   

        reviewerName                                         reviewText  \
0             Monika                                        Bad quality   
1            LovelyB               Sizing was

## Item-Item based CF Recommendation

In [30]:
ratings_active.head()

,reviewerID,asin,overall
70,A3TLFFGO88RXWZ,B00007GDFV,5
114,AAQO19HKS86MQ,B00008JOQI,4
124,A3126KT1D7PTXK,B00008JOQI,4
302,A17EV3FU9TH83L,B00062NHH0,4
306,A3QEUMXKRKQ4HF,B00062NHH0,1


In [31]:
# subsampling 1
SAMPLING_RATE = 1/2

# Downsample by users
user_id_unique = ratings_active.reviewerID.unique()
user_id_sample = pd.DataFrame(user_id_unique, columns=['unique_user_id']).sample(frac= SAMPLING_RATE, replace=False, random_state=1)

ratings_sample_50 = ratings_active.merge(user_id_sample, left_on='reviewerID', right_on='unique_user_id').drop(['unique_user_id'], axis=1)
print(ratings_sample_50.head())
print(ratings_sample_50.shape)

       reviewerID        asin  overall
0  A3QEUMXKRKQ4HF  B00062NHH0        1
1  A3QEUMXKRKQ4HF  B00WER0M96        5
2  A3QEUMXKRKQ4HF  B01535OKZM        1
3  A2PE67CSRSEJT1  B00062NHH0        5
4  A2PE67CSRSEJT1  B012Z17FZE        5
(33260, 3)


In [32]:
n_users = len(ratings_sample_50.reviewerID.unique())
n_recipe = len(ratings_sample_50.asin.unique())
print('Unique Users: {0}, unique item: {1}'.format(n_users, n_recipe))

Unique Users: 8786, unique item: 19152


In [33]:
user_item_m_50 = ratings_sample_50.pivot('reviewerID','asin','overall').fillna(0)
print(f'Shape: {user_item_m_50.shape}')

Shape: (8786, 19152)


In [34]:
user_item_m_50.head()

asin,1291691480,B00008JQLL,B00009Z70E,B0000WWTY8,B0000WWUMO,B000100E96,B00012O2RY,B00017WRK8,B00019XRWI,B0001F331C,...,B01HIRGLUU,B01HIWA9CQ,B01HIZYZ1O,B01HJ0SF82,B01HJB30HC,B01HJEO9SS,B01HJEOBCC,B01HJEOBUO,B01HJEOC9E,B01HJG5NLI
reviewerID,,,,,,,,,,,,,,,,,,,,,
A00248626G99Y8GVGKKQ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A03375698TKTJ5B1TCV3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A0985541ZZH41ONSQ0Q8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A09878118KLIMMUD7ERM,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A10036437T54UNY5JRYU,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
# cosine similarity between items
from sklearn.metrics.pairwise import cosine_similarity
X_items = cosine_similarity(user_item_m_50.T) # item similarity matrix

In [36]:
X_items.shape

(19152, 19152)

In [37]:
def item_based(item, similar, top_rank):
    # find the recipe_id and the corresponding coloumn
    locate = user_item_m_50.columns.get_loc(item)
    sim = X_items[locate]
    # get the most similar neigbours
    most_similar = user_item_m_50.columns[sim.argpartition(-(similar+1))[-(similar+1):]]
    # return the top rank list.
    return (most_similar.difference([item]).to_frame().reset_index(drop=True).head(top_rank))

In [39]:
h = item_based('B01HIRGLUU', 50, 10)
h

,asin
0,B00OH6EOGO
1,B00OHW2E9C
2,B00OI2EGHY
3,B00OIAW7YU
4,B00OIOG5CG
5,B00OIPYNGU
6,B00OIRYNS6
7,B00OIS3WDC
8,B00OISXQTW
9,B00OISXX2W


## Model based CF Recommendation

In [43]:
!pip install scikit-surprise

In [44]:
import pandas as pd

from surprise import Dataset, NormalPredictor, Reader
from surprise.model_selection import cross_validate

reader = Reader(rating_scale=(1, 5))

data = Dataset.load_from_df(df[["reviewerID", "asin", "overall"]], reader)

In [45]:
from surprise import SVD

algo = SVD()

# Run 5-fold cross-validation and print results.
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.3217  1.3218  1.3193  1.3190  1.3235  1.3211  0.0017  
MAE (testset)     1.0724  1.0717  1.0697  1.0691  1.0734  1.0713  0.0016  
Fit time          54.68   55.25   54.77   54.83   54.96   54.90   0.20    
Test time         2.52    1.39    1.39    1.38    1.40    1.62    0.45    


{'test_rmse': array([1.32173384, 1.32181659, 1.31930123, 1.31901273, 1.32351538]),
 'test_mae': array([1.07239453, 1.07166363, 1.06974479, 1.06913116, 1.07340837]),
 'fit_time': (54.68492531776428,
  55.2464063167572,
  54.76999807357788,
  54.829761266708374,
  54.959481954574585),
 'test_time': (2.518049478530884,
  1.392185926437378,
  1.388037919998169,
  1.3754985332489014,
  1.4014840126037598)}

In [16]:
R = np.array(user_item_active)
user_ratings_mean = np.mean(R, axis = 1)

In [17]:
R_demeaned = R - user_ratings_mean.reshape(-1, 1)

In [18]:
from scipy import linalg                                                                            
from scipy.sparse import linalg as slinalg      
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(R_demeaned, k = 50)

In [19]:
U.shape

(17573, 50)

In [20]:
sigma.shape

(50,)

In [21]:
s = np.diag(sigma)

In [22]:
s.shape

(50, 50)

In [23]:
Vt.shape

(50, 32235)

In [24]:
all_user_predicted_ratings = np.dot(np.dot(U, s), Vt) + user_ratings_mean.reshape(-1, 1)
preds_rating = pd.DataFrame(all_user_predicted_ratings, columns = user_item_active.columns)


In [25]:
preds_rating

asin,1291691480,7106116521,B00007GDFV,B00008JOQI,B00008JPRZ,B00008JQLL,B00009Z70E,B0000E62XO,B0000EVXVQ,B0000WWTY8,...,B01HJ2SAEE,B01HJAQ8OK,B01HJB30HC,B01HJEO9SS,B01HJEOBCC,B01HJEOBUO,B01HJEOC9E,B01HJFD866,B01HJG5NLI,B01HJHTH5U
0,-0.001122,0.003857,-0.000075,-0.001526,-0.000019,-0.000068,-0.000001,-0.000066,-0.000064,-0.000074,...,-0.000046,-0.000064,-0.000053,-0.000060,-0.000743,-0.000427,-0.000210,-0.000164,-0.000034,-0.000067
1,0.000086,0.000093,0.000093,0.000122,0.000091,0.000093,0.000091,0.000093,0.000093,0.000093,...,0.000093,0.000093,0.000093,0.000093,0.000092,0.000092,0.000079,0.000093,0.000094,0.000093
2,0.000308,0.000312,0.000316,0.000297,0.000307,0.000316,0.000310,0.000316,0.000316,0.000316,...,0.000316,0.000316,0.000316,0.000316,0.000314,0.000315,0.000304,0.000316,0.000316,0.000316
3,0.000340,0.000342,0.000348,0.000327,0.000339,0.000348,0.000341,0.000348,0.000348,0.000348,...,0.000348,0.000348,0.000348,0.000347,0.000345,0.000346,0.000337,0.000347,0.000347,0.000348
4,0.000402,0.000405,0.000411,0.000386,0.000400,0.000411,0.000403,0.000411,0.000411,0.000411,...,0.000411,0.000411,0.000411,0.000411,0.000408,0.000409,0.000398,0.000411,0.000411,0.000411
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17568,0.000175,0.002663,-0.000007,0.001791,0.000031,0.000002,-0.000081,0.000004,0.000006,-0.000006,...,0.000027,0.000006,0.000018,0.000007,-0.000690,-0.000764,-0.000177,-0.000226,-0.000009,-0.000001
17569,0.000007,0.000086,0.000053,0.000751,0.000558,0.000057,0.000038,0.000058,0.000060,0.000053,...,0.000070,0.000059,0.000065,0.000062,0.000032,0.000003,-0.000075,0.000075,0.000063,0.000056
17570,0.000097,0.000090,0.000091,0.000099,0.000088,0.000091,0.000089,0.000091,0.000091,0.000091,...,0.000091,0.000091,0.000091,0.000091,0.000095,0.000094,0.000083,0.000090,0.000091,0.000091
17571,0.000555,0.000560,0.000570,0.000535,0.000554,0.000570,0.000557,0.000570,0.000569,0.000570,...,0.000569,0.000569,0.000569,0.000569,0.000565,0.000566,0.000552,0.000569,0.000569,0.000569


In [26]:
def recommend():
    n = input("Enter User ID : ")
    a = preds_rating.iloc[n,:]
    a = pd.DataFrame(a)
    a['recipe_id'] = user_item_m_50.columns
    a = a.reset_index(drop=True)
    name = a.columns
    name = name[0]
    a = a.rename(columns = {name:'id'})
    a = a.sort_values(by='id',ascending=[False])
    print(list(a.iloc[:10,1]))

In [42]:
recommend()

,asin
0,B000GHMRLW
1,B000GHRZN2
2,B00OHW2E9C
3,B00OI2EGHY
4,B00OIAW7YU
5,B00OIOG5CG
6,B00OIPYNGU
7,B00OIRYNS6
8,B00OIS3WDC
9,B00OISXQTW


Citation:

Ups and downs: Modeling the visual evolution of fashion trends with one-class collaborative filtering R. He, J. McAuley WWW, 2016 pdf

Image-based recommendations on styles and substitutes J. McAuley, C. Targett, J. Shi, A. van den Hengel SIGIR, 2015 pdf

@article{Hug2020, doi = {10.21105/joss.02174}, url = {https://doi.org/10.21105/joss.02174}, year = {2020}, publisher = {The Open Journal}, volume = {5}, number = {52}, pages = {2174}, author = {Nicolas Hug}, title = {Surprise: A Python library for recommender systems}, journal = {Journal of Open Source Software} }